In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import re


In [2]:
df=pd.read_csv('F:/python for DS/train.csv')
#droping those features which do not contribute to survival
df=df.drop(['PassengerId','Ticket','Name'],axis=1)

In [3]:
#creating two more feature
#1. relative =SibSp+Parch
#2.individual= yes=0/ no=1
df['relative']=df['SibSp']+df['Parch']
df.loc[df['relative']>0,'individual']=0
df.loc[df['relative']==0,'individual']=1
df['individual']=df['individual'].astype(int)


In [4]:
#coverting cabin name into numeric according to their alphabetical order
#e.g. a/A=1, b/=2
alpha={'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}

df['Cabin']=df['Cabin'].fillna('u0')
df['newCabin']=df['Cabin'].map(lambda x:re.compile("([a-zA-Z])").search(x).group())
df['newCabin']=df['newCabin'].map(alpha)
df['newCabin']=df['newCabin'].fillna(0)
df['newCabin']=df['newCabin'].astype(int)

df=df.drop(['Cabin'],axis=1)

In [5]:
#filling empty rows in age column with mean of all ages
fill_value=df['Age'].mean(skipna=True)
df['Age']=df['Age'].fillna(fill_value)
df['Age']=df['Age'].astype(int)

#convert age into  different  intervals
df.loc[df['Age']<=16 ,'Age']=0
df.loc[(df['Age']>16) & (df['Age']<=32), 'Age']=1
df.loc[(df['Age']>32) & (df['Age']<=48), 'Age']=2
df.loc[(df['Age']>48) & (df['Age']<=64), 'Age']=3
df.loc[df['Age']>64 , 'Age']=4



In [6]:
#filling empty rows in embarked by their most occuring value

most_occured='S'
df['Embarked']=df['Embarked'].fillna(most_occured)
embark={'S':1,'C':2,'Q':3}
df['Embarked']=df['Embarked'].map(embark)


In [7]:
#coverting fare float value to int and 0 to empty rows
df['Fare']=df['Fare'].astype(int)
df['Fare']=df['Fare'].fillna(0)


In [8]:
#coverting gender value into integers
gender={'male':0,'female':1}
df['Sex']=df['Sex'].map(gender)


In [9]:
target=df['Survived']
df=df.drop(['Survived','SibSp','Parch','newCabin'],axis=1)

x_train,x_test,y_train,y_test=train_test_split(df,target,test_size=0.30,random_state=42)
print(df.head(10))

   Pclass  Sex  Age  Fare  Embarked  relative  individual
0       3    0    1     7         1         1           0
1       1    1    2    71         2         1           0
2       3    1    1     7         1         0           1
3       1    1    2    53         1         1           0
4       3    0    2     8         1         0           1
5       3    0    1     8         3         0           1
6       1    0    3    51         1         0           1
7       3    0    0    21         1         4           0
8       3    1    1    11         1         2           0
9       2    1    0    30         2         1           0


In [10]:
model=DecisionTreeClassifier(max_depth=3,splitter='best',criterion='entropy')
model.fit(x_train,y_train)
print(model)
y_pred=model.predict(x_test)
dt=tree.export_text(model)
print(dt)
print("accuracy for depth {}:".format(3),metrics.accuracy_score(y_test,y_pred)*100)

DecisionTreeClassifier(criterion='entropy', max_depth=3)
|--- feature_1 <= 0.50
|   |--- feature_2 <= 0.50
|   |   |--- feature_5 <= 4.00
|   |   |   |--- class: 1
|   |   |--- feature_5 >  4.00
|   |   |   |--- class: 0
|   |--- feature_2 >  0.50
|   |   |--- feature_0 <= 1.50
|   |   |   |--- class: 0
|   |   |--- feature_0 >  1.50
|   |   |   |--- class: 0
|--- feature_1 >  0.50
|   |--- feature_0 <= 2.50
|   |   |--- feature_3 <= 148.50
|   |   |   |--- class: 1
|   |   |--- feature_3 >  148.50
|   |   |   |--- class: 1
|   |--- feature_0 >  2.50
|   |   |--- feature_3 <= 23.50
|   |   |   |--- class: 1
|   |   |--- feature_3 >  23.50
|   |   |   |--- class: 0

accuracy for depth 3: 81.34328358208955
